#### Importing packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import regularizers, Sequential, layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model, save_model
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasClassifier  # Import KerasClassifier
from sklearn.model_selection import cross_validate
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import cross_validate

#### Upload dataset

In [2]:
data = pd.read_csv('../raw_data/validation_dataset_1.csv')
data.head()

having_IP_Address  URL_Length  Shortining_Service  having_At_Symbol  \
0                  1           1                   0                 0   
1                  0           1                   0                 0   
2                  0           0                   0                 0   
3                  0           0                   0                 0   
4                  0           0                   1                 0   

   double_slash_redirecting  Prefix_Suffix  having_Sub_Domain  SSLfinal_State  \
0                         1             -1                 -1              -1   
1                         0             -1                  0               1   
2                         0             -1                 -1              -1   
3                         0             -1                 -1              -1   
4                         0             -1                  1               1   

   Domain_registeration_length  Favicon  ...  popUpWidnow  Iframe  \
0                            0        0  ...            0       0   
1                            0        0  ...            0       0   
2                            0        0  ...            0       0   
3                            1        0  ...            0       0   
4                            0        0  ...            1       0   

   age_of_domain  DNSRecord  web_traffic  Page_Rank  Google_Index  \
0             -1          1           -1         -1             0   
1             -1          1            0         -1             0   
2              0          1            1         -1             0   
3             -1          1            1         -1             0   
4             -1          1            0         -1             0   

   Links_pointing_to_page  Statistical_report  Result  
0                       1                   1       1  
1                       1                   0       1  
2                       0                   1       1  
3                      -1                   0       1  
4                       1                   0       0  

[5 rows x 31 columns]

#### Preprocessing data

##### Drop irrelevant columns

In [3]:
data.columns

Index(['having_IP_Address', 'URL_Length', 'Shortining_Service',
       'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix',
       'having_Sub_Domain', 'SSLfinal_State', 'Domain_registeration_length',
       'Favicon', 'port', 'HTTPS_token', 'Request_URL', 'URL_of_Anchor',
       'Links_in_tags', 'SFH', 'Submitting_to_email', 'Abnormal_URL',
       'Redirect', 'on_mouseover', 'RightClick', 'popUpWidnow', 'Iframe',
       'age_of_domain', 'DNSRecord', 'web_traffic', 'Page_Rank',
       'Google_Index', 'Links_pointing_to_page', 'Statistical_report',
       'Result'],
      dtype='object')

In [4]:
data.drop(columns=['Statistical_report', 'web_traffic', 'Page_Rank', 'Abnormal_URL', 'DNSRecord', 'URL_Length', 'Domain_registeration_length'], inplace=True)

In [5]:
data.columns

Index(['having_IP_Address', 'Shortining_Service', 'having_At_Symbol',
       'double_slash_redirecting', 'Prefix_Suffix', 'having_Sub_Domain',
       'SSLfinal_State', 'Favicon', 'port', 'HTTPS_token', 'Request_URL',
       'URL_of_Anchor', 'Links_in_tags', 'SFH', 'Submitting_to_email',
       'Redirect', 'on_mouseover', 'RightClick', 'popUpWidnow', 'Iframe',
       'age_of_domain', 'Google_Index', 'Links_pointing_to_page', 'Result'],
      dtype='object')

In [7]:
indices_to_remove = data[data['Result'] == 0].index

indices_to_remove = indices_to_remove[:300]

data.drop(indices_to_remove, inplace=True)

#### Define X and y

In [9]:
X = data.drop(columns=['Result'])
y = data['Result']

#### Split data on train and test sets

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=42, stratify=y)

In [14]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1509, 23), (647, 23), (1509,), (647,))

#### Initialize the model

In [15]:
def initialize_model():
    
    reg_l1 = regularizers.L1(0.02)
    # reg_l2 = regularizers.L2(0.02)
    
    model = Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(23,)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(32, activation='relu', bias_regularizer=reg_l1))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    opt = tf.keras.optimizers.Adam(learning_rate=0.00009)
    model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
    return model

#### Fitting the model

In [16]:
model = initialize_model()

es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train, 
          epochs=100, batch_size=32, 
          verbose=1, validation_data=(X_test, y_test))

Epoch 1/100


2023-12-23 19:00:14.596337: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


48/48 [==============================] - 0s 2ms/step - loss: 0.7636 - accuracy: 0.5017 - val_loss: 0.7305 - val_accuracy: 0.4992
Epoch 2/100
48/48 [==============================] - 0s 608us/step - loss: 0.7264 - accuracy: 0.5083 - val_loss: 0.6955 - val_accuracy: 0.5100
Epoch 3/100
48/48 [==============================] - 0s 592us/step - loss: 0.6897 - accuracy: 0.5209 - val_loss: 0.6668 - val_accuracy: 0.5487
Epoch 4/100
48/48 [==============================] - 0s 624us/step - loss: 0.6635 - accuracy: 0.5613 - val_loss: 0.6405 - val_accuracy: 0.5981
Epoch 5/100
48/48 [==============================] - 0s 601us/step - loss: 0.6379 - accuracy: 0.6057 - val_loss: 0.6151 - val_accuracy: 0.6337
Epoch 6/100
48/48 [==============================] - 0s 632us/step - loss: 0.6111 - accuracy: 0.6561 - val_loss: 0.5888 - val_accuracy: 0.7125
Epoch 7/100
48/48 [==============================] - 0s 620us/step - loss: 0.5833 - accuracy: 0.7044 - val_loss: 0.5616 - val_accuracy: 0.7743
Epoch 8/100
4

#### Testing the model

In [17]:
test_accuracy = model.evaluate(X_test, y_test)[1]

21/21 [==============================] - 0s 416us/step - loss: 0.1581 - accuracy: 0.9397


#### Save the model

In [18]:
from tensorflow.keras.models import load_model, save_model

model.save(f'../saved_models/validation_model_1.h5')